# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa09c3f2f40>
├── 'a' --> tensor([[ 0.6856,  1.0648, -0.8241],
│                   [ 0.5070, -0.5600, -0.2682]])
└── 'x' --> <FastTreeValue 0x7fa09c3f2e80>
    └── 'c' --> tensor([[-1.3735, -0.6888,  1.2909,  1.3938],
                        [ 0.5157,  1.1300,  0.6155, -0.0728],
                        [ 0.4535,  1.7566, -0.3956,  1.2623]])

In [4]:
t.a

tensor([[ 0.6856,  1.0648, -0.8241],
        [ 0.5070, -0.5600, -0.2682]])

In [5]:
%timeit t.a

64.2 ns ± 0.0837 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa09c3f2f40>
├── 'a' --> tensor([[-0.4164,  0.4732, -2.2995],
│                   [ 1.3633,  0.5062,  0.8104]])
└── 'x' --> <FastTreeValue 0x7fa09c3f2e80>
    └── 'c' --> tensor([[-1.3735, -0.6888,  1.2909,  1.3938],
                        [ 0.5157,  1.1300,  0.6155, -0.0728],
                        [ 0.4535,  1.7566, -0.3956,  1.2623]])

In [7]:
%timeit t.a = new_value

87.9 ns ± 0.156 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6856,  1.0648, -0.8241],
               [ 0.5070, -0.5600, -0.2682]]),
    x: Batch(
           c: tensor([[-1.3735, -0.6888,  1.2909,  1.3938],
                      [ 0.5157,  1.1300,  0.6155, -0.0728],
                      [ 0.4535,  1.7566, -0.3956,  1.2623]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6856,  1.0648, -0.8241],
        [ 0.5070, -0.5600, -0.2682]])

In [11]:
%timeit b.a

68.9 ns ± 7.77 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1456,  0.5943, -0.4166],
               [-0.6058,  0.7533, -0.2882]]),
    x: Batch(
           c: tensor([[-1.3735, -0.6888,  1.2909,  1.3938],
                      [ 0.5157,  1.1300,  0.6155, -0.0728],
                      [ 0.4535,  1.7566, -0.3956,  1.2623]]),
       ),
)

In [13]:
%timeit b.a = new_value

586 ns ± 69.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

964 ns ± 50.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.8 µs ± 778 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

166 µs ± 4.83 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

166 µs ± 7.51 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9fce6aadc0>
├── 'a' --> tensor([[[ 0.6856,  1.0648, -0.8241],
│                    [ 0.5070, -0.5600, -0.2682]],
│           
│                   [[ 0.6856,  1.0648, -0.8241],
│                    [ 0.5070, -0.5600, -0.2682]],
│           
│                   [[ 0.6856,  1.0648, -0.8241],
│                    [ 0.5070, -0.5600, -0.2682]],
│           
│                   [[ 0.6856,  1.0648, -0.8241],
│                    [ 0.5070, -0.5600, -0.2682]],
│           
│                   [[ 0.6856,  1.0648, -0.8241],
│                    [ 0.5070, -0.5600, -0.2682]],
│           
│                   [[ 0.6856,  1.0648, -0.8241],
│                    [ 0.5070, -0.5600, -0.2682]],
│           
│                   [[ 0.6856,  1.0648, -0.8241],
│                    [ 0.5070, -0.5600, -0.2682]],
│           
│                   [[ 0.6856,  1.0648, -0.8241],
│                    [ 0.5070, -0.5600, -0.2682]]])
└── 'x' --> <FastTreeValue 0x7f9fce6aadf0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

41 µs ± 6.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9fce6aac70>
├── 'a' --> tensor([[ 0.6856,  1.0648, -0.8241],
│                   [ 0.5070, -0.5600, -0.2682],
│                   [ 0.6856,  1.0648, -0.8241],
│                   [ 0.5070, -0.5600, -0.2682],
│                   [ 0.6856,  1.0648, -0.8241],
│                   [ 0.5070, -0.5600, -0.2682],
│                   [ 0.6856,  1.0648, -0.8241],
│                   [ 0.5070, -0.5600, -0.2682],
│                   [ 0.6856,  1.0648, -0.8241],
│                   [ 0.5070, -0.5600, -0.2682],
│                   [ 0.6856,  1.0648, -0.8241],
│                   [ 0.5070, -0.5600, -0.2682],
│                   [ 0.6856,  1.0648, -0.8241],
│                   [ 0.5070, -0.5600, -0.2682],
│                   [ 0.6856,  1.0648, -0.8241],
│                   [ 0.5070, -0.5600, -0.2682]])
└── 'x' --> <FastTreeValue 0x7f9fce6aac40>
    └── 'c' --> tensor([[-1.3735, -0.6888,  1.2909,  1.3938],
                        [ 0.5157,  1.1300,  0.6155, -0.0728],
                 

In [23]:
%timeit t_cat(trees)

35.5 µs ± 7.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

76.8 µs ± 6.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.6856,  1.0648, -0.8241],
                [ 0.5070, -0.5600, -0.2682]],
       
               [[ 0.6856,  1.0648, -0.8241],
                [ 0.5070, -0.5600, -0.2682]],
       
               [[ 0.6856,  1.0648, -0.8241],
                [ 0.5070, -0.5600, -0.2682]],
       
               [[ 0.6856,  1.0648, -0.8241],
                [ 0.5070, -0.5600, -0.2682]],
       
               [[ 0.6856,  1.0648, -0.8241],
                [ 0.5070, -0.5600, -0.2682]],
       
               [[ 0.6856,  1.0648, -0.8241],
                [ 0.5070, -0.5600, -0.2682]],
       
               [[ 0.6856,  1.0648, -0.8241],
                [ 0.5070, -0.5600, -0.2682]],
       
               [[ 0.6856,  1.0648, -0.8241],
                [ 0.5070, -0.5600, -0.2682]]]),
    x: Batch(
           c: tensor([[[-1.3735, -0.6888,  1.2909,  1.3938],
                       [ 0.5157,  1.1300,  0.6155, -0.0728],
                       [ 0.4535,  1.7566, -0.3956,  1.2623]],
         

In [26]:
%timeit Batch.stack(batches)

94.9 µs ± 6.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.6856,  1.0648, -0.8241],
               [ 0.5070, -0.5600, -0.2682],
               [ 0.6856,  1.0648, -0.8241],
               [ 0.5070, -0.5600, -0.2682],
               [ 0.6856,  1.0648, -0.8241],
               [ 0.5070, -0.5600, -0.2682],
               [ 0.6856,  1.0648, -0.8241],
               [ 0.5070, -0.5600, -0.2682],
               [ 0.6856,  1.0648, -0.8241],
               [ 0.5070, -0.5600, -0.2682],
               [ 0.6856,  1.0648, -0.8241],
               [ 0.5070, -0.5600, -0.2682],
               [ 0.6856,  1.0648, -0.8241],
               [ 0.5070, -0.5600, -0.2682],
               [ 0.6856,  1.0648, -0.8241],
               [ 0.5070, -0.5600, -0.2682]]),
    x: Batch(
           c: tensor([[-1.3735, -0.6888,  1.2909,  1.3938],
                      [ 0.5157,  1.1300,  0.6155, -0.0728],
                      [ 0.4535,  1.7566, -0.3956,  1.2623],
                      [-1.3735, -0.6888,  1.2909,  1.3938],
                      [ 0.5157,  

In [28]:
%timeit Batch.cat(batches)

173 µs ± 6.79 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

396 µs ± 49.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
